## groupBy()

- groupBy  : 집계함수를 가지고 있는 GroupData 객체를 반환한다.  

- GrouopData객체의 집계함수들을 사용해 grouping 된 데이터들의 집계결과를 저장하고 있는 DataFrame을 반환 받을 수 있다.

In [6]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [7]:
from datetime import date, datetime
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import functions as F

In [8]:
cdf = spark.read.csv("/dataframe/a_class_info.csv", header=True)
cdf.printSchema()

root
 |-- class_cd: string (nullable = true)
 |-- school: string (nullable = true)
 |-- class_std_cnt: string (nullable = true)
 |-- loc: string (nullable = true)
 |-- school_type: string (nullable = true)
 |-- teaching_type: string (nullable = true)



In [9]:
# 지역별 교육타입별 학생 숫자를 구해보자.
# cdf.groupby(cdf.loc, cdf.teaching_type).sum('class_std_cnt').show()
# AnalysisException: "class_std_cnt" is not a numeric column.
# class_std_cnt가 문자열이라 문제가 된다.
# 스키마를 적용해서 해결 해보자

In [10]:
# 스키마 설정
schema = StructType([
    StructField("class_cd", StringType()),
    StructField("school", StringType()),
    StructField("class_std_cnt", IntegerType()),
    StructField("loc", StringType()),
    StructField("school_type", StringType()),
    StructField("teaching_type", StringType()),
])

In [13]:
cdf = spark.read.csv("/dataframe/a_class_info.csv", header=True, schema=schema)
cdf.printSchema()
type(cdf)

root
 |-- class_cd: string (nullable = true)
 |-- school: string (nullable = true)
 |-- class_std_cnt: integer (nullable = true)
 |-- loc: string (nullable = true)
 |-- school_type: string (nullable = true)
 |-- teaching_type: string (nullable = true)



pyspark.sql.dataframe.DataFrame

In [15]:
# 지역별 교육타입별 학생 숫자를 구해보자.
cdf.groupby(cdf.loc, cdf.teaching_type).sum("class_std_cnt").show()

+--------+-------------+------------------+
|     loc|teaching_type|sum(class_std_cnt)|
+--------+-------------+------------------+
|   Rural| Experimental|               211|
|    NULL|         NULL|              NULL|
|   Urban|     Standard|               631|
|Suburban|     Standard|               433|
|   Rural|     Standard|               327|
|Suburban| Experimental|               284|
|   Urban| Experimental|               275|
+--------+-------------+------------------+



- sql.df.where("조건문자열" or 조건 표현식)

In [24]:
print('지역별 class 숫자를 계산해보자. 단 지역정보가 없는 데이터는 제외한다.')
cdf.where(cdf.loc.isNotNull()) \
    .groupby(cdf.loc) \
    .count().show() # count : group dataset 객체 내 집계 합수

cdf.where(cdf.loc.isNotNull()) \
    .groupby(cdf.loc) \
    .agg(count("class_cd")).show() # count : sql.function의 count 집계 함수

지역별 class 숫자를 계산해보자. 단 지역정보가 없는 데이터는 제외한다.
+--------+-----+
|     loc|count|
+--------+-----+
|   Urban|   37|
|Suburban|   34|
|   Rural|   28|
+--------+-----+

+--------+---------------+
|     loc|count(class_cd)|
+--------+---------------+
|   Urban|             37|
|Suburban|             34|
|   Rural|             28|
+--------+---------------+



In [54]:
print('지역별 교육타입별 학생 숫자와 평균을 구해보자. 단 학생 숫자가 300미만인 데이터는 제외한다.')
cdf.groupby(cdf.loc, cdf.teaching_type) \
    .agg(sum("class_std_cnt"), avg("class_std_cnt")) \
    .where(sum("class_std_cnt") >= 300).show()

# cdf.groupby(cdf.loc, cdf.teaching_type) \
#     .agg(sum("class_std_cnt"), avg("class_std_cnt")) \
#     .where(col("sum(class_std_cnt)") >= 300).show()

print('컬럼명이 sum(class_std_cnt) 이라니 너무 이상하다. 집계함수를 수행하고 별칭을 붙여보자')
cdf.groupby(cdf.loc, cdf.teaching_type) \
    .agg(sum("class_std_cnt").alias("최댓값"), 
         avg("class_std_cnt").alias("평균")) \
         .where(sum("class_std_cnt") >= 300).show()

지역별 교육타입별 학생 숫자와 평균을 구해보자. 단 학생 숫자가 300미만인 데이터는 제외한다.
+--------+-------------+------------------+------------------+
|     loc|teaching_type|sum(class_std_cnt)|avg(class_std_cnt)|
+--------+-------------+------------------+------------------+
|   Urban|     Standard|               631| 24.26923076923077|
|Suburban|     Standard|               433|             21.65|
|   Rural|     Standard|               327|           20.4375|
+--------+-------------+------------------+------------------+

컬럼명이 sum(class_std_cnt) 이라니 너무 이상하다. 집계함수를 수행하고 별칭을 붙여보자
+--------+-------------+------+-----------------+
|     loc|teaching_type|최댓값|             평균|
+--------+-------------+------+-----------------+
|   Urban|     Standard|   631|24.26923076923077|
|Suburban|     Standard|   433|            21.65|
|   Rural|     Standard|   327|          20.4375|
+--------+-------------+------+-----------------+



In [44]:
cdf.columns

['class_cd', 'school', 'class_std_cnt', 'loc', 'school_type', 'teaching_type']

In [72]:
# 학교가 가장 많이 위치한 지역의 학생 수 총합과, 가장 적게 위치한 지역의 학생 수 총 합 간의 차이를 구해보자
# cdf.groupby(cdf.loc) \
#     .agg(count("school")).show()
cdf.groupby(cdf.loc) \
    .agg(count("school"),
        sum("class_std_cnt").alias("tot")).show()

cdf.groupby(cdf.loc) \
    .agg(count("school"),
        sum("class_std_cnt").alias("tot")) \
    .where(cdf.loc.isNotNull()) \
    .select((max(col('tot')) - min(col('tot'))).alias("학생수 차이")).show()

+--------+-------------+----+
|     loc|count(school)| tot|
+--------+-------------+----+
|   Urban|           37| 906|
|    NULL|            0|NULL|
|Suburban|           34| 717|
|   Rural|           26| 538|
+--------+-------------+----+

+-----------+
|학생수 차이|
+-----------+
|        368|
+-----------+



#### sql

In [62]:
# dataFrame을 view테이블로 등록
cdf.createOrReplaceTempView("classV")

In [ ]:
print('지역별 class 숫자를 계산해보자. 단 지역정보가 없는 데이터는 제외한다.')
# count("*") : record를 세기때문에 행의 모든 값이 null이 아닌이상 count
spark.sql('''
    select loc, count("*")
    from classV
    group by loc
    having loc is not null    
''').show()
# count("특정열") : 특정열의 값을 세기때문에 값이 null인 경우 안세줌
spark.sql('''
    select loc, count("class_cd") as count
    from classV
    group by loc
    having loc is not null    
''').show()

In [69]:
print('''지역내 교육타입별 학생 숫자와 평균을 구해보자. 
단  지역내 교육타입별 학생 숫자의 총 합이 300미만인 데이터는 제외한다.''')   
spark.sql('''
    select loc, teaching_type, sum(class_std_cnt), avg(class_std_cnt)
    from classV
    group by loc, teaching_type
    having sum(class_std_cnt) >= 300
''').show()

print('컬럼명이 sum(class_std_cnt) 이라니 너무 이상하다. 집계함수를 수행하고 별칭을 붙여보자')

지역내 교육타입별 학생 숫자와 평균을 구해보자. 
단  지역내 교육타입별 학생 숫자의 총 합이 300미만인 데이터는 제외한다.
+--------+-------------+------------------+------------------+
|     loc|teaching_type|sum(class_std_cnt)|avg(class_std_cnt)|
+--------+-------------+------------------+------------------+
|   Urban|     Standard|               631| 24.26923076923077|
|Suburban|     Standard|               433|             21.65|
|   Rural|     Standard|               327|           20.4375|
+--------+-------------+------------------+------------------+

컬럼명이 sum(class_std_cnt) 이라니 너무 이상하다. 집계함수를 수행하고 별칭을 붙여보자


## orderBy()

In [ ]:
print('반 학생 숫자를 기준으로 내림차순 정렬하라')
cdf.orderBy(cdf.class_std_cnt.desc()).show(2)
print('loc를 기준으로 오름차순 정렬하라, 이때 같은 지역끼리는 학교이름을 기준으로 내림차순 정렬하라')
cdf.orderBy(cdf.loc.asc(), cdf.school.desc()).show(2)

In [92]:
# 연습
# null이 가장 작은수로 판단
# 내림차순 : 가장 아래로
# 오름차순 : null을 가장 먼저
print('학교 종류를 기준으로 오름차순 정렬하라, 만약 school_type이 null인 행이 있다면 제일 위로 오게 하라')
cdf.orderBy(cdf.school_type.asc()).show(4)
cdf.orderBy(cdf.school_type.desc()).show(4)
cdf.orderBy(cdf.school_type.desc_nulls_first()).show(4)
cdf.orderBy(cdf.school_type.asc_nulls_last()).show()

학교 종류를 기준으로 오름차순 정렬하라, 만약 school_type이 null인 행이 있다면 제일 위로 오게 하라
+--------+------+-------------+-----+-----------+-------------+
|class_cd|school|class_std_cnt|  loc|school_type|teaching_type|
+--------+------+-------------+-----+-----------+-------------+
|     6PP|  NULL|         NULL| NULL|       NULL|         NULL|
|     4SZ|  NULL|         NULL| NULL|       NULL|         NULL|
|     5SD|  NULL|         NULL| NULL|       NULL|         NULL|
|     6OL| ANKYI|           20|Urban| Non-public|     Standard|
+--------+------+-------------+-----+-----------+-------------+
only showing top 4 rows

+--------+------+-------------+--------+-----------+-------------+
|class_cd|school|class_std_cnt|     loc|school_type|teaching_type|
+--------+------+-------------+--------+-----------+-------------+
|     1Q1| CUQAM|           28|   Urban|     Public|     Standard|
|     2AP| DNQDD|           27|Suburban|     Public|     Standard|
|     BFY| CUQAM|           27|   Urban|     Public|     Standar

#### sql

In [84]:
print('반 학생 숫자를 기준으로 내림차순 정렬하라')
spark.sql('''
    select *
    from classV
    order by class_std_cnt desc
''').show()
print('loc를 기준으로 오름차순 정렬하라, 이때 같은 지역끼리는 학교이름을 기준으로 내림차순 정렬하라')
spark.sql('''
    select *
    from classV
    order by loc asc, school desc
''').show()

반 학생 숫자를 기준으로 내림차순 정렬하라
+--------+------+-------------+--------+-----------+-------------+
|class_cd|school|class_std_cnt|     loc|school_type|teaching_type|
+--------+------+-------------+--------+-----------+-------------+
|     18K| GOOBU|           31|   Urban|     Public|     Standard|
|     A93| VVTVA|           30|   Urban|     Public| Experimental|
|     YTB| VVTVA|           30|   Urban|     Public| Experimental|
|     Q0E| ZOWMK|           30|   Urban|     Public| Experimental|
|     QA2| ZOWMK|           30|   Urban|     Public|     Standard|
|     ZBH| ZOWMK|           30|   Urban|     Public|     Standard|
|     7BL| VVTVA|           29|   Urban|     Public|     Standard|
|     1Q1| CUQAM|           28|   Urban|     Public|     Standard|
|     OMI| CUQAM|           28|   Urban|     Public|     Standard|
|     ROP| DNQDD|           28|Suburban|     Public| Experimental|
|     HKF| GOOBU|           28|   Urban|     Public|     Standard|
|     0N7| QOQTS|           28|   Urba

In [94]:
# 연습
print('학교 종류를 기준으로 오름차순 정렬하라, 만약 school_type이 null인 행이 있다면 제일 위로 오게 하라')
spark.sql('''
    select *
    from classV
    order by school_type is null, school_type asc
''').show()

학교 종류를 기준으로 오름차순 정렬하라, 만약 school_type이 null인 행이 있다면 제일 위로 오게 하라
+--------+------+-------------+--------+-----------+-------------+
|class_cd|school|class_std_cnt|     loc|school_type|teaching_type|
+--------+------+-------------+--------+-----------+-------------+
|     6OL| ANKYI|           20|   Urban| Non-public|     Standard|
|     ZNS| ANKYI|           21|   Urban| Non-public|     Standard|
|     2B1| CCAAW|           18|Suburban| Non-public| Experimental|
|     EPS| CCAAW|           20|Suburban| Non-public| Experimental|
|     IQN| CCAAW|           15|Suburban| Non-public| Experimental|
|     PGK| CCAAW|           21|Suburban| Non-public|     Standard|
|     UHU| CCAAW|           16|Suburban| Non-public| Experimental|
|     UWK| CCAAW|           19|Suburban| Non-public|     Standard|
|     A33| CIMBB|           19|   Urban| Non-public|     Standard|
|     EID| CIMBB|           21|   Urban| Non-public|     Standard|
|     HUJ| CIMBB|           17|   Urban| Non-public| Experimental